In [1]:
import glob
import os
from shutil import rmtree

In [2]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [3]:
def next_smaller_base2_int(number):
    nsb = 1
    while nsb < number:
        nsb *= 2
    return nsb/2

# Train OpenCV cascade classifier
See [cascade training documentation]( http://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html#cascade-training).

In [4]:
obj = 'duplo_brick'

In [5]:
if os.name == 'posix':
    code_dirname = os.path.join(u'/home', 'baxter', 'ros_ws', 'src')
elif os.name == 'nt':
    code_dirname = os.path.join(u'\\\srv-file.brml.tum.de', 'nthome', 'mludersdorfer', 'reallyMyDocuments', 'code')
else:
    print 'wtf?'

package_dirname = os.path.join(code_dirname, 'baxter_pick_and_place')
print package_dirname

\\srv-file.brml.tum.de\nthome\mludersdorfer\reallyMyDocuments\code\baxter_pick_and_place


In [6]:
data = os.path.join(package_dirname, 'data', 'synthetic_demo_data')
vec = os.path.join(data, '%s_20_20.vec' % obj)
bg = os.path.join(data, 'bg.txt')
num_stages = 17
precalc_val_buf_size = 256
precalc_idx_buf_size = 256
acceptance_ratio_break_value = 10e-5
feature_type = 'HAAR'
w = 20
h = 20
bt = 'GAB'
min_hit_rate = 0.999
max_false_alarm_rate = 0.001
weight_trim_rate = 0.95
max_depth = 1
max_weak_count = 150
mode = 'ALL'

num_neg = file_len(bg)
num_vec = file_len(os.path.join(data, '%s_info.dat' % obj))
num_pos = (num_vec - num_neg)/(1+(num_stages-1)*(1-min_hit_rate))
num_pos = next_smaller_base2_int(num_pos)
print "Train '%s'-classifier using %i pos. and %i neg. samples." % \
    (obj, num_pos, num_neg)

Train 'duplo_brick'-classifier using 512 pos. and 75 neg. samples.


In [7]:
cmd = ('opencv_traincascade ' + 
       '-data %s ' % data +
       '-vec %s ' % vec + 
       '-bg %s ' % bg + 
       '-numPos %i ' % num_pos + 
       '-numNeg %i ' % num_neg + 
       '-numStages %i ' % num_stages + 
       '-precalcValBufSize %i ' % precalc_val_buf_size + 
       '-precalcIdxBufSize %i ' % precalc_idx_buf_size + 
       '-acceptanceRatioBreakValue %f ' % acceptance_ratio_break_value + 
       '-featureType %s ' % feature_type + 
       '-w %i ' % w + 
       '-h %i ' % h + 
       '-bt %s ' % bt + 
       '-minHitRate %f ' % min_hit_rate + 
       '-maxFalseAlarmRate %f ' % max_false_alarm_rate +
       '-weightTrimRate %f ' % weight_trim_rate + 
       '-maxDepth %i ' % max_depth + 
       '-maxWeakCount %i ' % max_weak_count +
       '-mode %s' % mode)
print '%s\n'% cmd
!{cmd}

opencv_traincascade -data \\srv-file.brml.tum.de\nthome\mludersdorfer\reallyMyDocuments\code\baxter_pick_and_place\data\synthetic_demo_data -vec \\srv-file.brml.tum.de\nthome\mludersdorfer\reallyMyDocuments\code\baxter_pick_and_place\data\synthetic_demo_data\duplo_brick_20_20.vec -bg \\srv-file.brml.tum.de\nthome\mludersdorfer\reallyMyDocuments\code\baxter_pick_and_place\data\synthetic_demo_data\bg.txt -numPos 512 -numNeg 75 -numStages 17 -precalcValBufSize 256 -precalcIdxBufSize 256 -acceptanceRatioBreakValue 0.000100 -featureType HAAR -w 20 -h 20 -bt GAB -minHitRate 0.999000 -maxFalseAlarmRate 0.001000 -weightTrimRate 0.950000 -maxDepth 1 -maxWeakCount 150 -mode ALL

---------------------------------------------------------------------------------
Training parameters are pre-loaded from the parameter file in data folder!
Please empty this folder if you want to use a NEW set of training parameters.
---------------------------------------------------------------------------------
PARAM

## Rename trained classifier

In [8]:
old = os.path.join(data, 'cascade.xml')
new = os.path.join(data, '%s_cascade.xml' % obj)
if os.path.exists(new):
    os.remove(new)
os.rename(old, new)

## Move temporary files out of the way

In [9]:
xml_dirname = os.path.join(data, 'temp_cascade', '%s' % obj)
if os.path.exists(xml_dirname):
    rmtree(xml_dirname)
os.makedirs(xml_dirname)

xml_files = glob.glob(os.path.join(data, '*.xml'))
xml_files = [xf for xf in xml_files if 'cascade' not in xf]
for xf in xml_files:
    new_xf = os.path.join(xml_dirname, xf.split('\\')[-1])
    os.rename(xf, new_xf)